In [4]:
import sys
import os
import numpy as np
import zipfile
from tqdm import tqdm
import scrapbook as sb
from tempfile import TemporaryDirectory
import tensorflow as tf
tf.get_logger().setLevel('ERROR') # only show error messages

from recommenders.models.deeprec.deeprec_utils import download_deeprec_resources
from recommenders.models.newsrec.newsrec_utils import prepare_hparams
from recommenders.models.newsrec.models.lstur import LSTURModel
from recommenders.models.newsrec.io.mind_iterator import MINDIterator
from recommenders.models.newsrec.newsrec_utils import get_mind_data_set

print("System version:{}".format(sys.version))
print("Tensorflow version:{}".format(tf.__version__))

System version:3.8.17 (default, Jul  5 2023, 20:44:21) [MSC v.1916 64 bit (AMD64)]
Tensorflow version:2.6.1


In [13]:
import numpy as np

In [3]:
!pip install keras==2.6

     ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
     ------- -------------------------------- 0.2/1.3 MB 5.0 MB/s eta 0:00:01
     ------------- -------------------------- 0.5/1.3 MB 4.7 MB/s eta 0:00:01
     --------------------- ------------------ 0.7/1.3 MB 4.9 MB/s eta 0:00:01
     ------------------------------- -------- 1.0/1.3 MB 5.4 MB/s eta 0:00:01
     ---------------------------------------  1.3/1.3 MB 5.8 MB/s eta 0:00:01
     ---------------------------------------- 1.3/1.3 MB 5.5 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.13.1
    Uninstalling keras-2.13.1:
      Successfully uninstalled keras-2.13.1


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.6.1 requires numpy~=1.19.2, but you have numpy 1.20.3 which is incompatible.


In [5]:
!pip install protobuf==3.20.* --user

In [3]:
!pip install scrapbook

  Obtaining dependency information for pyarrow from https://files.pythonhosted.org/packages/17/65/47c519fd198713631ca004e38ae48e244a4f4708ba32691caa54b36508a4/pyarrow-13.0.0-cp38-cp38-win_amd64.whl.metadata
  Obtaining dependency information for tenacity from https://files.pythonhosted.org/packages/f4/f1/990741d5bb2487d529d20a433210ffa136a367751e454214013b441c4575/tenacity-8.2.3-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/24.4 MB ? eta -:--:--
    --------------------------------------- 0.4/24.4 MB 8.3 MB/s eta 0:00:03
   - -------------------------------------- 0.7/24.4 MB 7.8 MB/s eta 0:00:04
   - -------------------------------------- 1.1/24.4 MB 8.1 MB/s eta 0:00:03
   -- ------------------------------------- 1.6/24.4 MB 8.6 MB/s eta 0:00:03
   --- ------------------------------------ 2.2/24.4 MB 9.3 MB/s eta 0:00:03
   --- ------------------------------------ 2.4/24.4 MB 8.9 MB/s eta 0:00:03
   ---- ----------------------------------- 2.8/24.4 MB 8.6 

In [4]:
!pip install tensorflow==2.6.1

In [8]:
!pip install numpy==1.20.3 --user

  Using cached numpy-1.20.3-cp38-cp38-win_amd64.whl (13.7 MB)


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.57.1 requires numpy<1.25,>=1.21, but you have numpy 1.20.3 which is incompatible.
tensorflow 2.6.1 requires numpy~=1.19.2, but you have numpy 1.20.3 which is incompatible.


In [1]:
conda uninstall numpy

Solving environment: ...working... failed

Note: you may need to restart the kernel to use updated packages.


DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): repo.anaconda.com:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): repo.anaconda.com:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): repo.anaconda.com:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): repo.anaconda.com:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): repo.anaconda.com:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): repo.anaconda.com:443
DEBUG:urllib3.connectionpool:https://repo.anaconda.com:443 "GET /pkgs/msys2/win-64/repodata.json HTTP/1.1" 304 0
DEBUG:urllib3.connectionpool:https://repo.anaconda.com:443 "GET /pkgs/msys2/noarch/repodata.json HTTP/1.1" 304 0
DEBUG:urllib3.connectionpool:https://repo.anaconda.com:443 "GET /pkgs/r/noarch/repodata.json HTTP/1.1" 304 0
DEBUG:urllib3.connectionpool:https://repo.anaconda.com:443 "GET /pkgs/r/win-64/repodata.json HTTP/1.1" 304 0
DEBUG:urllib3.connectionpool:https://rep

In [1]:
!pip show numpy

Name: numpy
Version: 1.20.3
Summary: NumPy is the fundamental package for array computing with Python.
Home-page: https://www.numpy.org
Author: Travis E. Oliphant et al.
Author-email: 
License: BSD
Location: c:\users\super\appdata\roaming\python\python38\site-packages
Requires: 
Required-by: Bottleneck, category-encoders, contourpy, cornac, h5py, Keras-Preprocessing, lightfm, lightgbm, matplotlib, numba, opt-einsum, pandas, pandera, patsy, powerlaw, pyarrow, recommenders, scikit-learn, scikit-surprise, scipy, seaborn, statsmodels, tensorboard, tensorflow, transformers


In [27]:
epochs=1
seed=40
bath_size=32

MIND_type='demo'

In [8]:
tmpdir = TemporaryDirectory()
data_path = tmpdir.name

train_news_file = os.path.join(data_path, 'train', r'news.tsv')
train_behaviors_file = os.path.join(data_path, 'train', r'behaviors.tsv')
valid_news_file = os.path.join(data_path, 'valid', r'news.tsv')
valid_behaviors_file = os.path.join(data_path, 'valid', r'behaviors.tsv')
wordEmb_file = os.path.join(data_path, "utils", "embedding.npy")
userDict_file = os.path.join(data_path, "utils", "uid2index.pkl")
wordDict_file = os.path.join(data_path, "utils", "word_dict.pkl")
yaml_file = os.path.join(data_path, "utils", r'lstur.yaml')

mind_url, mind_train_dataset, mind_dev_dataset, mind_utils = get_mind_data_set(MIND_type)

if not os.path.exists(train_news_file):
    download_deeprec_resources(mind_url, os.path.join(data_path, 'train'), mind_train_dataset)
    
if not os.path.exists(valid_news_file):
    download_deeprec_resources(mind_url, \
                               os.path.join(data_path, 'valid'), mind_dev_dataset)
if not os.path.exists(yaml_file):
    download_deeprec_resources(r'https://recodatasets.z20.web.core.windows.net/newsrec/', \
                               os.path.join(data_path, 'utils'), mind_utils)

100%|██████████| 17.0k/17.0k [00:03<00:00, 4.52kKB/s]
100%|██████████| 9.84k/9.84k [00:01<00:00, 6.32kKB/s]
100%|██████████| 95.0k/95.0k [00:27<00:00, 3.46kKB/s]


In [29]:
print(yaml_file)

C:\Users\super\AppData\Local\Temp\tmpfecpj2nm\utils\lstur.yaml


In [9]:
hparams = prepare_hparams(yaml_file, 
                          wordEmb_file=wordEmb_file,
                          wordDict_file=wordDict_file, 
                          userDict_file=userDict_file,
                          batch_size=batch_size,
                          epochs=epochs)
print(hparams)

HParams object with values {'support_quick_scoring': True, 'dropout': 0.2, 'attention_hidden_dim': 200, 'head_num': 4, 'head_dim': 100, 'filter_num': 400, 'window_size': 3, 'vert_emb_dim': 100, 'subvert_emb_dim': 100, 'gru_unit': 400, 'type': 'ini', 'user_emb_dim': 50, 'learning_rate': 0.0001, 'optimizer': 'adam', 'epochs': 1, 'batch_size': 32, 'show_step': 100000, 'title_size': 30, 'his_size': 50, 'data_format': 'news', 'npratio': 4, 'metrics': ['group_auc', 'mean_mrr', 'ndcg@5;10'], 'word_emb_dim': 300, 'cnn_activation': 'relu', 'model_type': 'lstur', 'loss': 'cross_entropy_loss', 'wordEmb_file': 'C:\\Users\\super\\AppData\\Local\\Temp\\tmpfecpj2nm\\utils\\embedding.npy', 'wordDict_file': 'C:\\Users\\super\\AppData\\Local\\Temp\\tmpfecpj2nm\\utils\\word_dict.pkl', 'userDict_file': 'C:\\Users\\super\\AppData\\Local\\Temp\\tmpfecpj2nm\\utils\\uid2index.pkl'}


In [28]:
iterator = MINDIterator

In [11]:
model = LSTURModel(hparams, iterator, seed=seed)

Tensor("conv1d/Relu:0", shape=(None, 30, 400), dtype=float32)
Tensor("att_layer2/Sum_1:0", shape=(None, 400), dtype=float32)


C:\Users\super\anaconda3\envs\python38\lib\site-packages\keras\optimizer_v2\optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


In [12]:
print(model.run_eval(valid_news_file, valid_behaviors_file))

0it [00:00, ?it/s]C:\Users\super\anaconda3\envs\python38\lib\site-packages\keras\engine\training.py:2470: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '
586it [00:15, 38.93it/s]
236it [04:11,  1.06s/it]
7538it [00:01, 4640.89it/s]


{'group_auc': 0.5201, 'mean_mrr': 0.2214, 'ndcg@5': 0.2292, 'ndcg@10': 0.2912}


In [13]:
%%time
model.fit(train_news_file, train_behaviors_file, valid_news_file, valid_behaviors_file)

1086it [13:16:04, 43.98s/it]  
586it [00:13, 42.55it/s]
236it [07:10,  1.83s/it]
7538it [00:02, 2741.25it/s]


at epoch 1
train info: logloss loss:1.4900762238994267
eval info: group_auc:0.5925, mean_mrr:0.2587, ndcg@10:0.3471, ndcg@5:0.2824
at epoch 1 , train time: 47764.4 eval time: 466.6
CPU times: total: 5h 59min 11s
Wall time: 13h 23min 50s


In [14]:
%%time
res_syn = model.run_eval(valid_news_file, valid_behaviors_file)
print(res_syn)

586it [00:12, 48.47it/s]
236it [03:50,  1.03it/s]
7538it [00:01, 4749.46it/s]


{'group_auc': 0.5925, 'mean_mrr': 0.2587, 'ndcg@5': 0.2824, 'ndcg@10': 0.3471}
CPU times: total: 12min 10s
Wall time: 4min 13s


In [15]:
sb.glue("res_syn", res_syn)

In [16]:
model_path = os.path.join(data_path, "model")
os.makedirs(model_path, exist_ok=True)

model.model.save_weights(os.path.join(model_path, "lstur_ckpt"))

In [17]:
model.model.save_weights("lstur_ckpt.h5")

In [18]:
group_impr_indexes, group_labels, group_preds = model.run_fast_eval(valid_news_file, valid_behaviors_file)

586it [00:12, 46.61it/s]
236it [03:52,  1.01it/s]
7538it [00:01, 4146.01it/s]


In [25]:
with open(os.path.join(data_path, 'prediction.txt'), 'w') as f:
    for impr_index, preds in tqdm(zip(group_impr_indexes, group_preds)):
        impr_index += 1
        pred_rank = (np.argsort(np.argsort(preds)[::-1]) + 1).tolist()
        pred_rank = '[' + ','.join([str(i) for i in pred_rank]) + ']'
        f.write(' '.join([str(impr_index), pred_rank])+ '\n')

7538it [00:00, 20153.79it/s]


In [26]:
f = zipfile.ZipFile(os.path.join(data_path, 'prediction.zip'), 'w', zipfile.ZIP_DEFLATED)
f.write(os.path.join(data_path, 'prediction.txt'), arcname='prediction.txt')
f.close()